In [0]:
# -*- coding: utf-8 -*-
"""first0.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/12wCE4-KOGy3UpH929Q0mrjAtBzvnGoov
"""

import nltk 
nltk.download('universal_tagset')
nltk.download('brown')
import numpy as np
import copy
import pandas as pd
import math
from tqdm import tqdm
from nltk.corpus import brown as corpus

tagged_words = [ ] #word with tag
all_tags = [ ]  #tags sequence 

my_set={"START","END"}
for sent in corpus.tagged_sents(tagset='universal'): # get tagged sentences
    tagged_words.append( ("START", "START") )
    all_tags.append("START")
    for (word, tag) in sent:
        all_tags.append(tag)
        tagged_words.append( (tag, word) ) 
        my_set.add(word)
    tagged_words.append( ("END", "END") )
    all_tags.append("END")

# print(*map(' '.join,nltk.bigrams(all_tags)),sep=' ,')

utagset = [
    'START',
    'VERB',
    'NOUN',
    'PRON',
    'ADJ',
    'ADV',
    'ADP',
    'CONJ',
    'DET',
    'NUM',
    'PRT',
    'X',
    '.',
    'END'
]

#calculate transition_prob
cfd_tags= nltk.ConditionalFreqDist(nltk.bigrams(all_tags))
transition_prob_df = pd.DataFrame(index=utagset, columns=utagset, data=np.zeros((len(utagset), len(utagset))))
transition_prob_df
rowi,colj=transition_prob_df.shape
for i in range(rowi):
    for j in range(colj):
         transition_prob_df.at[utagset[i],utagset[j] ]=(cfd_tags[utagset[i]][utagset[j]])/all_tags.count(utagset[i])


#calculate emission_prob
ls_col=list(my_set)
cfd_tagwords= nltk.ConditionalFreqDist(tagged_words)
cpd_tagwords = nltk.ConditionalProbDist(cfd_tagwords, nltk.MLEProbDist)
emission_prob_df = pd.DataFrame(index=utagset, columns=ls_col, data=np.zeros((len(utagset), len(my_set))))
rowx,coly=emission_prob_df.shape
emission_prob_df
for i in tqdm(range(int(rowx))):
    for j in range(int(coly)):
#         cpd_tagwords['DET'].prob('the') 
#         val=(cfd_tagwords[utagset[i]][ls_col[j]])/all_tags.count(utagset[i])
        emission_prob_df.at[utagset[i],ls_col[j] ]= cpd_tagwords[utagset[i]].prob(ls_col[j]) 



[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


100%|██████████| 14/14 [00:11<00:00,  1.61it/s]


what is example of complex sentence
DET
VERB
NOUN
ADP
ADJ
NOUN


In [0]:
		
#take input sentence from user to test
s=input().split()
utags=[
	'VERB',
    'NOUN',
    'PRON',
    'ADJ',
    'ADV',
    'ADP',
    'CONJ',
    'DET',
    'NUM',
    'PRT',
    'X',
    '.']

#calculate final probability list by multipying transition_prob and emission_prob across layer 
big_list=list()
best_path=list() #list track previous tag index
best_path_val=list()
for sidx in range(len(s)):
    first_list=list()
    sub_path=list()
    sub_path_val=list()
    for tidx in range(len(utags)):
        small_list=list()
        if (sidx==0):
            p=(transition_prob_df.loc['START'][utags[tidx]])*(emission_prob_df.loc[utags[tidx]][s[sidx]])
            first_list.append(p)
           
        else:
            for jdx in range(len(utags)):
                p=(big_list[sidx-1][jdx])*(transition_prob_df.loc[utags[jdx]][utags[tidx]])*(emission_prob_df.loc[utags[tidx]][s[sidx]])
                small_list.append(p)
            sub_path.append(small_list.index(max(small_list)))
            sub_path_val.append(max(small_list))
    
    
    if(sidx==0):
      best_path_val.append(max(first_list))
      big_list.append(first_list)
    else:
      best_path.append(sub_path)
      best_path_val.append(sub_path_val)
      big_list.append(sub_path_val)

#deepcopy of big_list 
big_copy_list=copy.deepcopy(big_list)
paths=list()

#func to check list contain probabilities of possible tag for last word is zero or not
def check1(list1): 
  for x in list1: 
     if x> 0: 
          return True 
  return False
 

#iterative code to find possible subsequence  
while(check1(big_copy_list[len(big_copy_list)-1])):

  idx=big_copy_list[len(big_copy_list)-1].index(max(big_copy_list[len(big_copy_list)-1]))

  max1=max(big_copy_list[len(big_copy_list)-1])

  dumpath=list()

  dumpath.append(idx)

  dumidx=len(best_path)-1
  inx=idx
  while(dumidx>=0):
    inx=best_path[dumidx][inx]
    dumpath.append(inx)
    dumidx-=1

  paths.append(dumpath)

  big_copy_list[len(big_copy_list)-1][idx]=0


#find best possible sequence by calculation of probabilities
max_p=list()
for slist in paths:
  mul=1
  k=len(big_list)-1
  for ix in slist:
    mul=big_list[k][ix]*mul
    k-=1
  max_p.append(mul)


#take max probability index 
f_idx=max_p.index(max(max_p))
paths[f_idx].reverse() #use max probability index for finding sequence in paths 


#print tag crosspendence to utag sequence
for i in paths[f_idx]:
  print(utags[i])

i am lucky
NOUN
VERB
ADJ
